In [1]:
### from imutils.video import VideoStream
import face_recognition
import argparse
import imutils
import pickle
import time
from datetime import datetime
import cv2
def markAttendance(name):
    with open("C:\\Users\\91809\\OneDrive\\Desktop\\project\\Attendence.csv",'r+') as f:
        mydatalist=f.readlines()
        namelist=[]
        for line in mydatalist:
            entry=line.split(',')
            namelist.append(entry[0])
        if name not in namelist:
            now=datetime.now()
            dtstring=now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtstring}')
def mail():
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders
    sender_email = "prasanthshadow93@gmail.com"
    receiver_email = "venkateswarlu192maguluri@gmail.com"
    message = MIMEMultipart()
    message["From"] = sender_email
    message['To'] = receiver_email
    message['Subject'] = "sending mail using python"
    file = "C:\\Users\\91809\\OneDrive\\Desktop\\project\\Attendence.csv"
    attachment = open(file,'rb')
    obj = MIMEBase('application','octet-stream')
    obj.set_payload((attachment).read())
    encoders.encode_base64(obj)
    obj.add_header('Content-Disposition',"attachment; filename= "+file)
    message.attach(obj)
    my_message = message.as_string()
    email_session = smtplib.SMTP('smtp.gmail.com',587)
    email_session.starttls()
    email_session.login(sender_email,'DevilShadow381251@&..')
    email_session.sendmail(sender_email,receiver_email,my_message)
    email_session.quit()
    print("YOUR MAIL HAS BEEN SENT SUCCESSFULLY")

print("[INFO] loading encodings...")
encoding="C:\\Users\\91809\\OneDrive\\Desktop\\project2\\encoding.plx"
output="C:\\Users\\91809\\OneDrive\\Desktop\\project2"
data = pickle.loads(open(encoding, "rb").read())
print("[INFO] starting video stream...")
cap=cv2.VideoCapture(0)
writer = None
while True:
    sucess,frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])
    boxes = face_recognition.face_locations(rgb,model='hog')
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"],encoding)
        name = "Unknown"
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            
        name = max(counts, key=counts.get)
        names.append(name)
    for((top, right, bottom, left), name) in zip(boxes, names):
        top = int(top * r)
        right = int(right * r)
        bottom = int(bottom * r)
        left = int(left * r)
        cv2.rectangle(frame, (left, top), (right, bottom),(0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
        markAttendance(name)
    if 1 > 0:
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
cv2.destroyAllWindows()
cv2.waitKey(1)
mail()
print("Done.....!")

[INFO] loading encodings...
[INFO] starting video stream...


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials ob6sm949943pjb.30 - gsmtp')